In [1]:
#james chartouni
import lightgbm as lgb
import xgboost as xgb
import pandas as pd
import numpy as np
from numpy import inf
from sklearn.decomposition import PCA, KernelPCA, TruncatedSVD
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis, QuadraticDiscriminantAnalysis
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.pipeline import make_pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler, RobustScaler, PolynomialFeatures
from sklearn.metrics import log_loss

In [2]:
training_data = pd.read_csv("cleaned_input/train_consolidated.csv")
training_data = training_data.drop(['msno'], axis=1)

In [24]:
test_data = pd.read_csv("cleaned_input/test_consolidated.csv")
test_data.shape
test_data = test_data.drop(['msno'], axis=1)
test_data.shape

(907471, 125)

In [25]:
X_eval = test_data.drop(['is_churn'], axis=1)
y_eval = test_data['is_churn']

In [4]:
#split data 
y = training_data["is_churn"].values
X = training_data.drop(["is_churn"], axis=1).values
#replaced infinite values with zero. MAKE SURE THIS IS the right thing to do 
X[X == -inf] = 0
X[X == inf] = 0

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=.15, shuffle=True)

In [14]:
#logistic regression 
logistic_pipe = make_pipeline(LogisticRegression(solver='liblinear'))
scores = cross_val_score(logistic_pipe, X_train, y_train, cv=5, scoring="neg_log_loss")
print("Accuracy: %0.2f (+/- %0.4f)" % (scores.mean(), scores.std() * 2))

#logistic regression w/ polynomial features 2   
logistic_pipe = make_pipeline(PolynomialFeatures(degree=2), LogisticRegression(solver='liblinear'))
scores = cross_val_score(logistic_pipe, X_train, y_train, cv=5, scoring="neg_log_loss")
print("Accuracy: %0.2f (+/- %0.4f)" % (scores.mean(), scores.std() * 2))

#logistic regression w/th PCA, Polynomial features 
logistic_pipe = make_pipeline(PCA(n_components=.95), PolynomialFeatures(degree=2), LogisticRegression(solver='liblinear'))
scores = cross_val_score(logistic_pipe, X_train, y_train, cv=5, scoring="neg_log_loss")
print("Accuracy: %0.2f (+/- %0.4f)" % (scores.mean(), scores.std() * 2))


Accuracy: -0.19 (+/- 0.1852)


/Users/James/anaconda/lib/python3.6/site-packages/sklearn/linear_model/base.py:340: RuntimeWarning: overflow encountered in exp
  np.exp(prob, prob)


Accuracy: -0.56 (+/- 0.5516)
Accuracy: -0.64 (+/- 0.2019)


/Users/James/anaconda/lib/python3.6/site-packages/sklearn/linear_model/base.py:340: RuntimeWarning: overflow encountered in exp
  np.exp(prob, prob)


In [26]:
#Light GBM 
# create dataset for lightgbm
# if you want to re-use data, remember to set free_raw_data=False
lgb_train = lgb.Dataset(X_train, y_train, free_raw_data=False)
lgb_eval = lgb.Dataset(X_test, y_test, reference=lgb_train,free_raw_data=False)


# specify your configurations as a dict
params = {
    'boosting_type': 'gbdt',
    'objective': 'binary',
    'metric': 'binary_logloss',
    'num_leaves': 16,
    'learning_rate': 0.01,
    'feature_fraction': 0.9,
    'bagging_fraction': 0.8,
    'bagging_freq': 5,
    'verbose': 0,
    'num_threads': 3
}
# generate a feature name
feature_name = X_train.dtype.names

#cross validation
num_round = 200
scores = lgb.cv(params, lgb_train, num_round, nfold=5, verbose_eval=True)

[1]	cv_agg's binary_logloss: 0.683205 + 7.02056e-07
[2]	cv_agg's binary_logloss: 0.674365 + 4.31011e-06
[3]	cv_agg's binary_logloss: 0.664793 + 4.01808e-06
[4]	cv_agg's binary_logloss: 0.656204 + 1.20761e-05
[5]	cv_agg's binary_logloss: 0.646983 + 1.18647e-05
[6]	cv_agg's binary_logloss: 0.637936 + 1.19441e-05
[7]	cv_agg's binary_logloss: 0.629837 + 1.97076e-05
[8]	cv_agg's binary_logloss: 0.621113 + 1.97492e-05
[9]	cv_agg's binary_logloss: 0.612552 + 1.97746e-05
[10]	cv_agg's binary_logloss: 0.60415 + 1.9074e-05
[11]	cv_agg's binary_logloss: 0.596638 + 2.45492e-05
[12]	cv_agg's binary_logloss: 0.588525 + 2.42318e-05
[13]	cv_agg's binary_logloss: 0.580557 + 2.38439e-05
[14]	cv_agg's binary_logloss: 0.572732 + 2.34625e-05
[15]	cv_agg's binary_logloss: 0.565045 + 2.27962e-05
[16]	cv_agg's binary_logloss: 0.557493 + 2.24839e-05
[17]	cv_agg's binary_logloss: 0.550073 + 2.2261e-05
[18]	cv_agg's binary_logloss: 0.542782 + 2.2046e-05
[19]	cv_agg's binary_logloss: 0.535616 + 2.17188e-05
[20]	c

In [22]:
model = lgb.train(params, lgb_train, num_boost_round=200)

#Prediction
y_pred=model.predict(X_test)


In [23]:
log_loss_score = log_loss(y_test, y_pred)
print(log_loss_score)

#convert into binary values
for i in range(0, len(y_pred)):
    if y_pred[i]>=.5:       # setting threshold to .5
        y_pred[i]=1
    else:  
        y_pred[i]=0

from sklearn.metrics import accuracy_score
accuracy=accuracy_score(y_pred,y_test)
print(accuracy)

0.0718077573718
0.999696261374


In [ ]:
print('Start training...')
# feature_name and categorical_feature
gbm = lgb.train(params,
                lgb_train,
                num_boost_round=50,
                valid_sets=lgb_train,  # eval training data
              )

In [ ]:
print("--------------------------------------------------------------")
ypred = gbm.predict(X_test)
print(log_loss(y_test, ypred))
# save model to file
gbm.save_model('model.txt')

# feature names
print('Feature names:', gbm.feature_name())

# feature importances
print('Feature importances:', list(gbm.feature_importance()))

In [23]:
from sklearn import metrics

def xgb_score(preds, dtrain):
    labels = dtrain.get_label()
    return 'log_loss', metrics.log_loss(labels, preds)

fold = 1
for i in range(fold):
    params = {
        'eta': 0.02, #use 0.002
        'max_depth': 7,
        'objective': 'binary:logistic',
        'eval_metric': 'logloss',
        'seed': i,
        'silent': False
    }
    watchlist = [(xgb.DMatrix(X_train, y_train), 'train'), (xgb.DMatrix(X_test, y_test), 'valid')]
    model = xgb.train(params, xgb.DMatrix(X_train, y_train), 150,  watchlist, feval=xgb_score, maximize=False, verbose_eval=50, early_stopping_rounds=50) #use 1500
    if i != 0:
        pred += model.predict(xgb.DMatrix(test[cols]), ntree_limit=model.best_ntree_limit)
    else:
        pred = model.predict(xgb.DMatrix(test[cols]), ntree_limit=model.best_ntree_limit)
pred /= fold
test['is_churn'] = pred.clip(0.0000001, 0.999999)
test[['msno','is_churn']].to_csv('submission3.csv.gz', index=False, compression='gzip')

[0]	train-logloss:0.673348	valid-logloss:0.673348	train-log_loss:0.673348	valid-log_loss:0.673348
Multiple eval metrics have been passed: 'valid-log_loss' will be used for early stopping.

Will train until valid-log_loss hasn't improved in 50 rounds.
[50]	train-logloss:0.197725	valid-logloss:0.197725	train-log_loss:0.197725	valid-log_loss:0.197725
[100]	train-logloss:0.06819	valid-logloss:0.06819	train-log_loss:0.06819	valid-log_loss:0.06819
[149]	train-logloss:0.025042	valid-logloss:0.025042	train-log_loss:0.025042	valid-log_loss:0.025042


NameError: name 'test' is not defined